# Titanic

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB

In [2]:
test = pd.read_csv("Data/test.csv")
train = pd.read_csv("Data/train.csv")

In [3]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Priprava podatkov

In [4]:
train = train[np.invert(np.isnan(train["Age"]))]
test["Age"] = test["Age"].fillna(np.mean(test["Age"]))
test["Fare"] = test["Fare"].fillna(np.mean(test["Fare"]))
#test = test[np.invert(np.isnan(test["Age"]))]
#test = test[np.invert(np.isnan(test["Fare"]))]

Nov atribut minor:

In [5]:
def minors(x):
    minor = []
    for a in x["Age"]:
        if a < 16:
            minor = np.append(minor, 1)
        else:
            minor = np.append(minor, 0)
    x["Minor"] = minor

In [6]:
minors(train)
minors(test)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


Nova atributa na podlagi priimkov:
* Vsaj en prezivel z istim priimkom
* Vsi umrli z istim priimkom

In [7]:
def allDied(x, y, ista):
    allD = []
    lived = []
    for index, row in x.iterrows():
        surname = row["Name"].split(",")[0]
        dead = 0
        alive = 0
        for index2, row2 in y.iterrows():
            if ista:
                if index != index2:
                    surname2 = row2["Name"].split(",")[0]
                    if surname2 == surname:
                        if row2["Survived"] == 1:
                            alive += 1
                        else:
                            dead += 1
            else:
                surname2 = row2["Name"].split(",")[0]
                if surname2 == surname:
                    if row2["Survived"] == 1:
                        alive += 1
                    else:
                        dead += 1
        if dead + alive < 2:
            allD = np.append(allD, 0)
            lived = np.append(lived, 0)
        else:
            if alive == 0:
                allD = np.append(allD, 1)
            else:
                allD = np.append(allD, 0)
            if alive >= 1:
                lived = np.append(lived, 1)
            else:
                lived = np.append(lived, 0)
    x["SurnameAllDied"] = allD
    x["SurnameOneLived"] = lived

In [8]:
allDied(train, train, True)
allDied(test, train, False)

## Pretvorba cabins

In [43]:
palube = []
for a in train["Cabin"]:
    if not pd.isna(a):
        if a[0] not in palube:
            palube = np.append(palube, a[0])

In [55]:
palube = []
for a in train["Cabin"]:
    if not pd.isna(a):
        palube = np.append(palube, a[0])
    else:
        palube = np.append(palube, a)
train["Deck"] = palube

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [56]:
palube = []
for a in test["Cabin"]:
    if not pd.isna(a):
        palube = np.append(palube, a[0])
    else:
        palube = np.append(palube, a)
test["Deck"] = palube

In [47]:
np.sort(palube)

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'], dtype='<U32')

Tabele za napovedovanje:

In [92]:
def one_hot(name, x, y):
    category = np.array([])
    for a in y[name]:
        if not pd.isna(a):
            if a not in category:
                category = np.append(category, a)
    category = np.sort(category)
    for c in category:
        if not pd.isna(c):
            current = []
            for a in y[name]:
                if a == c:
                    current = np.append(current, 1)
                else:
                    current = np.append(current, 0)
            x[c] = current

In [76]:
y = train["Survived"]
x = train[["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SibSp", "Parch"]]
x["FamilySize"] = train["Parch"] + train["SibSp"]
one_hot("Sex", x, train)
one_hot("Deck", x, train)
#one_hot("Embarked", x, train)
x = x.loc[:, x.columns != 'nan']
#enc = OneHotEncoder()
#enc.fit(train)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/tmp/ipykernel_4589/3114588019.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in category:


In [90]:
xtest = test[["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SibSp", "Parch"]]
xtest["FamilySize"] = test["Parch"] + test["SibSp"]
one_hot("Sex", xtest, test)
one_hot("Deck", xtest, test)
xtest = xtest.loc[:, xtest.columns != 'nan']
#one_hot("Embarked", xtest, test)

male
female
nan
B
E
A
C
D
F
G
/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/tmp/ipykernel_4589/1133658929.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in category:


## Ocenjevanje atributov

## Napovedovanje

### Random forest

In [162]:
x3 = x[["Minor", "SurnameAllDied", "female"]]
x3test = xtest[["Minor", "SurnameAllDied", "female"]]
clf = RandomForestClassifier()
clf.fit(x3,y)
r = clf.predict(x3test)

In [163]:
resoult = test[["PassengerId"]]
resoult["Survived"] = r
#resoult.to_csv("Predictions/rfc3.csv",  index=False)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


### Naive Bayes

In [77]:
gnb = GaussianNB()
gnb.fit(x,y)
tt = gnb.predict(xtest)

In [78]:
resoult2 = test[["PassengerId"]]
resoult2["Survived"] = tt
resoult2 == resoult
#resoult2.to_csv("Predictions/nbc2.csv",  index=False)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,PassengerId,Survived
0,True,True
1,True,False
2,True,True
3,True,False
4,True,False
...,...,...
413,True,True
414,True,True
415,True,True
416,True,True


## Napovedovanje + testiranje

### Random forest

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from itertools import combinations

In [22]:
maxS = 0.65
maxT = np.array([])
for s in range(4,15):
    for a in combinations(["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "FamilySize", "female", "male", "A", "B", "C", "D", "E", "F"], s):
        x1 = x[np.array(a)]
        ca = 0
        for i in range(7):
            x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.33)
            clf = RandomForestClassifier()
            clf.fit(x_train, y_train)
            r = clf.predict(x_test)
            ca += accuracy_score(y_test, r)
        current = ca / 7
        if maxS < current:
            maxS = current
            maxT = a
        #print(current)
    print(s)
    print(maxT)
    print(maxS)
    maxS = 0.65
    maxT = np.array([])

4
('Minor', 'SurnameAllDied', 'female', 'male')
0.8365617433414043
5
('Minor', 'SurnameAllDied', 'SurnameOneLived', 'female', 'male')
0.8371670702179177
6
('Minor', 'SurnameAllDied', 'SurnameOneLived', 'FamilySize', 'female', 'male')
0.8329297820823245
7
('Pclass', 'Age', 'Fare', 'Minor', 'SurnameAllDied', 'female', 'male')
0.8153753026634382
8
('Pclass', 'Age', 'Fare', 'Minor', 'SurnameAllDied', 'SurnameOneLived', 'female', 'male')
0.8123486682808717
9
('Pclass', 'Age', 'Fare', 'Minor', 'SurnameAllDied', 'SurnameOneLived', 'FamilySize', 'female', 'male')
0.8020581113801454


In [140]:
maxS

0.8401937046004841

### Naive bayes

In [93]:
maxS = 0.65
maxT = np.array([])
for s in range(4,15):
    for a in combinations(["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "FamilySize", "female", "male", "A", "B", "C", "D", "E", "F"], s):
        x1 = x[np.array(a)]
        ca = 0
        for i in range(10):
            x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.33)
            gnb = GaussianNB()
            gnb.fit(x_train, y_train)
            r = gnb.predict(x_test)
            ca += accuracy_score(y_test, r)
        current = ca / 10
        if maxS < current:
            maxS = current
            maxT = a
        #print(current)
    print(s)
    print(maxT)
    print(maxS)
    maxS = 0.65
    maxT = np.array([])

4
('Minor', 'SurnameAllDied', 'male', 'E')
0.8338983050847457
5
('Minor', 'SurnameAllDied', 'female', 'male', 'E')
0.8364406779661018
6
('Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'E')
0.8300847457627117
7
('Pclass', 'Age', 'Minor', 'SurnameAllDied', 'female', 'male', 'F')
0.8389830508474576
8
('Pclass', 'Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'E')
0.8360169491525424
9
('Age', 'Fare', 'Minor', 'SurnameAllDied', 'SurnameOneLived', 'female', 'male', 'A', 'E')
0.827542372881356
10
('Pclass', 'Age', 'Minor', 'SurnameAllDied', 'SurnameOneLived', 'female', 'male', 'A', 'B', 'E')
0.827542372881356
11
('Age', 'Minor', 'SurnameAllDied', 'SurnameOneLived', 'FamilySize', 'female', 'male', 'A', 'D', 'E', 'F')
0.8237288135593219
12
('Pclass', 'Age', 'Minor', 'SurnameAllDied', 'SurnameOneLived', 'FamilySize', 'female', 'male', 'A', 'D', 'E', 'F')
0.8165254237288135
13
('Pclass', 'Age', 'Fare', 'Minor', 'SurnameAllDied', 'SurnameOneLived', 'FamilySize', 'fema

In [26]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Minor,SurnameAllDied,SurnameOneLived
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.0,0.0,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.0,0.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.0,0.0,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q,0.0,1.0,0.0
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0.0,0.0,0.0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0.0,0.0,1.0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0.0,0.0,0.0
